In [2]:
from scipy.io import loadmat
import numpy as np
import math, itertools, copy
mat_variables = loadmat("matlab_param/test_5_5_2.mat")

In [3]:
def manhattan(p1, p2):
    ''' Manhattan distance'''
    x = abs(p1[0] - p2[0])**2
    y = abs(p1[1] - p2[1])**2
    return math.sqrt(x+y)

In [4]:
class Route:
    def __init__(self,idx,route):
        self.id = idx
        self.x, self.y, self.z = self.extractRoute(route)
        self.length = len(self.x)

    def extractRoute(self, route):
        x = route['x'][0][0][0][:]
        y = route['y'][0][0][0][:]
        z = route['z'][0][0][0][:]
        x_list, y_list, z_list = [],[],[]
        for j in range(len(x)):
            x_list.append(x[j])
            y_list.append(y[j])
            z_list.append(z[j])
        return x_list,y_list,z_list

class World:
    def __init__(self, fileName):
        # Global variable
        mat_variables = loadmat(fileName)
        self.map = mat_variables['map4']
        self.map_mini = mat_variables['map']
        self.numDrones = mat_variables['numRobots'][0][0]
        self.routes = [Route(i,mat_variables['routes'][0][i]) for i in range(self.numDrones)]
        self.maxLength = self.findMaxLength()
        # self.roots = [i[0].tolist() for i in mat_variables['R4'][0]]
        self.roots = [[self.routes[i].x[0], self.routes[i].y[0], self.routes[i].z[0]] for i in range(self.numDrones)]

        # Technical parameters
        self.z_land = 0.2 # target for landing
        self.z_target = 1 # normal fly altitude
        self.z_offset = 0.3 # altitude offset when conflicts
        
        # Solve conflicts
        # self.solveConflicts()

    def findMaxLength(self):
        ''' Find the path with maximum length '''
        temp = 0
        for i in range(len(self.routes)):
            max_i = max(len(self.routes[i].x),len(self.routes[i].y),len(self.routes[i].z))
            if max_i > temp:
                temp = max_i
        return temp

    def adjustLength(self):
        ''' Make paths of same length -> when occurred, bring the drones to land altitude'''
        for r in range(self.numDrones):
            while len(self.routes[r].x) < self.maxLength:
                self.routes[r].x.append(self.routes[r].x[-1]) # x: append last element
                self.routes[r].y.append(self.routes[r].y[-1]) # y: append last element
                self.routes[r].z.append(self.z_land) # z: append z land
            self.routes[r].length = self.maxLength

    def checkNextXYloc(self,k,idx):
        ''' Check conflict in only in x and y -> used when we changed altitude '''
        # k: next drone
        # idx: index next step in path
        for i in range(self.numDrones):
            if i != k:
                if (self.routes[k].x[idx] == self.routes[i].x[idx]
                    and self.routes[k].y[idx] == self.routes[i].y[idx]):
                    return True
        return False
    
    def conflictXY(self,j,l,idx):
        ''' Check conflict in X Y'''
        # j: current drone
        # l: next drone
        # idx: current index in path
        if (self.routes[j].x[idx] == self.routes[l].x[idx] 
            and self.routes[j].y[idx] == self.routes[l].y[idx]):
            return True
        else: return False
    
    def conflictXYZ(self,j,k,idx):
        ''' Check conflict in X Y Z '''
        # j: current drone
        # k: next drone
        # idx: current index in path
        if (self.routes[j].x[idx] == self.routes[k].x[idx]
            and self.routes[j].y[idx] == self.routes[k].y[idx]
            and self.routes[j].z[idx] == self.routes[k].z[idx]):
            return True
        else: return False

    def insertWait(self,id,idx):
        ''' Wait for 1 step'''
        # l: drone
        self.routes[id].x.insert(idx,self.routes[id].x[i-1])
        self.routes[id].y.insert(idx,self.routes[id].y[i-1])
        self.routes[id].z.insert(idx,self.routes[id].z[i-1])

    def copyLastZ(self,idx):
        ''' Copy value of previous Z for all drones'''
        for i in range(self.numDrones):
            if self.routes[i].z[idx] != self.z_land:
                self.routes[i].z[idx] = self.routes[i].z[idx-1]

    def setZtoTarget(self,idx,solvedConflict):
        ''' Bring back the drones to the z_target when possible'''
        for i in range(len(self.routes)):
            if i not in solvedConflict:
                if self.routes[i].z[idx] != self.z_land:
                    self.routes[i].z[idx] = self.z_target

    def countConflictXY(self,j,idx):
        ''' Count number of conflict between the current and the following drones'''
        droneID = []
        for i in range(j+1,self.numDrones):
            if self.conflictXY(j, i, idx):
                droneID.append(i)
        return droneID
        
    def solveConflicts(self):
        ''' Solve conflicts until no new steps are added'''
        self.adjustLength()
        prev_length = self.maxLength
        while True:
            for i in range(1,self.maxLength):

                # Copy the last Z for all drones
                self.copyLastZ(i)
                # Keep track of the conflict solved
                solvedConflict = []

                for j in range(self.numDrones):
                    # Z value of current drone
                    temp1 = self.routes[j].z[i]

                    # Find drone in conflict
                    droneInConflict = self.countConflictXY(j,i)
                    if droneInConflict:
                        solvedConflict.insert(0,j)
                    for el in droneInConflict:
                        if el in solvedConflict:
                            droneInConflict.remove(el)
                        else:
                            solvedConflict.append(el)
            
                    # Solve conflicts
                    for count in range(len(droneInConflict)):
                        if temp1 > self.z_target:
                            self.routes[droneInConflict[count]].z[i] = round(temp1 - (count+1)*self.z_offset,1)
                        elif temp1 < self.z_target:
                            self.routes[droneInConflict[count]].z[i] = round(temp1 + (count+1)*self.z_offset,1)
                        # If more thant 3 drones want to access the same cell, from the 4-th and on they have to wait
                        elif count > 1:
                            self.insertWait(droneInConflict[count],i)
                        elif count % 2 == 0: # temp1 == z_target
                            self.routes[droneInConflict[count]].z[i] = round(temp1 + self.z_offset,1)
                        else:
                            self.routes[droneInConflict[count]].z[i] = round(temp1 - self.z_offset,1)
            
                self.setZtoTarget(i,solvedConflict)

            # Check length
            self.adjustLength()
            self.maxLength = self.findMaxLength()
            if prev_length == self.maxLength:
                break

In [8]:
global routes
param = World("matlab_param/test_6_6_2.mat")
routes = param.routes

class route:
    def __init__(self,x,y,z):
        self.x, self.y, self.z = x,y,z

landed = [False]*param.numDrones
# x1 = [1,2,3,4]
# y1 = [5,6,7,8]
# x2 = [11,2,3,14,15]
# y2 = [16,6,7,19,20]
# z1 = [1,1,1,1]
# z2 = [1,1,1,1,1]
# x3 = [4,2,6,7]
# y3 = [1,6,1,1]
# z3 = [1,1,1,1]
# a = route(x1,y1,z1)
# b = route(x2,y2,z2)
# c = route(x3,y3,z3)
# routes = [a,b,c]
# route.x = x1
# route.y = y1
# route.z = z1
# route.x = x2
# route.y = y2
# route.z = z2
# route.x = x3
# route.y = y3
# route.z = z3

# for route in routes:
#     print(route.x)
#     print(route.y)

def conflictXY(routes,idx):
    ''' Check conflict in X Y'''
    listConflictID = []
    listConflictPos = []
    for i in range(len(routes)-1):
        flag = False
        for j in range(i+1,len(routes)):
            if (idx < len(routes[i].x)
                and idx < len(routes[j].x)
                and routes[i].x[idx] == routes[j].x[idx] 
                and routes[i].y[idx] == routes[j].y[idx]
                and [routes[i].x[idx],routes[i].y[idx]] not in listConflictPos):
                flag = True
                # Insert the wait in the drones which has the shortest routes
                if len(routes[i].x) <= len(routes[j].x):
                    listConflictID.append(i)
                else:
                    listConflictID.append(j)
        # Keep track of already seen conflict to avoid repetition
        if flag:
            listConflictPos.append([routes[i].x[0],routes[i].y[0]])
    return listConflictID

def insertWait(i,idx):
    ''' Wait for 1 step'''
    global routes
    routes[i].x.insert(idx, routes[i].x[idx-1])
    routes[i].y.insert(idx, routes[i].y[idx-1])
    routes[i].z.insert(idx, routes[i].z[idx-1])

def stopCondition():
    global landed
    if False in landed:
        return False
    else:
        return True

def help_goTo(idx):
    global routes, landed

    listConflictID = conflictXY(routes,idx)
    for i in range(len(routes)):
        # If alrady landed, do nothing
        if landed[i] == True:
            print("\nDrone{} already landed".format(i))
            continue
        # If index NOT within the routes, land
        elif not(idx < len(routes[i].x)):
            landed[i] = True
            print("\nDrone{} terminated his path -> landin".format(i))
            continue
        # Otherwise, check for conflict
        if i in listConflictID:
            print("\n--- Found conflict ---")
            insertWait(i,idx)
            print("\nDrone{} has to wait in {} ".format(i,[(routes[i].x[idx],routes[i].y[idx],routes[i].z[idx])]))
            continue

        goal = [(routes[i].x[idx], routes[i].y[idx], routes[i].z[idx])]
        pose = np.array(goal).astype(float)
        print("\nDrone{} goint towards {}".format(i, goal))
        # help_updateMap(cf.pose, cf.id)

idx = 0
while not(stopCondition()):
    help_goTo(idx)
    idx += 1
print("\n------concluded in step {}".format(idx-1))



Drone0 goint towards [(1.5, 2.5, 1)]

Drone1 goint towards [(4.5, 3.5, 1)]

Drone0 goint towards [(1.5, 3.0, 1)]

Drone1 goint towards [(4.0, 3.5, 1)]

Drone0 goint towards [(2.0, 3.0, 1)]

Drone1 goint towards [(3.5, 3.5, 1)]

Drone0 goint towards [(2.5, 3.0, 1)]

Drone1 goint towards [(3.0, 3.5, 1)]

Drone0 goint towards [(3.0, 3.0, 1)]

Drone1 goint towards [(2.5, 3.5, 1)]

Drone0 goint towards [(3.5, 3.0, 1)]

Drone1 goint towards [(2.0, 3.5, 1)]

Drone0 goint towards [(4.0, 3.0, 1)]

Drone1 goint towards [(2.0, 3.0, 1)]

Drone0 goint towards [(4.0, 2.5, 1)]

Drone1 goint towards [(2.0, 2.5, 1)]

Drone0 goint towards [(3.5, 2.5, 1)]

Drone1 goint towards [(1.5, 2.5, 1)]

Drone0 goint towards [(3.0, 2.5, 1)]

Drone1 goint towards [(1.5, 3.0, 1)]

Drone0 goint towards [(2.5, 2.5, 1)]

Drone1 goint towards [(1.5, 3.5, 1)]

Drone0 goint towards [(2.0, 2.5, 1)]

Drone1 goint towards [(1.5, 4.0, 1)]

Drone0 goint towards [(2.0, 2.0, 1)]

Drone1 goint towards [(1.5, 4.5, 1)]

Drone0 goin